In [3]:
import os
import rasterio
import numpy as np
from typing import List
import matplotlib.pyplot as plt
from scipy.stats import linregress as lr
import math as mt
from matplotlib.backends.backend_pdf import PdfPages
from datetime import datetime

In [7]:
def box_count(raster: np.ndarray, box_size: int):
    mask = ~(raster == -9999)
    # print(mask)
    rows, cols = raster.shape
    total_box_count = 0 
    box_count_for_size = 0
    for i in range(0, rows, box_size):
        for j in range(0, cols, box_size):
            end_row = min(i + box_size, rows)
            end_col = min(j + box_size, cols)
            # print(f"For location {i}, {j}, end row is {end_row} and end col is {end_col}")
            current_slice = raster[i:end_row, j:end_col]
            current_mask = mask[i:end_row, j:end_col]

            valid_vals = current_slice[current_mask]
            if valid_vals.size > 0:
                max_val = np.max(valid_vals)
                min_val = np.min(valid_vals)
                # print(f"max_val = {max_val}, min_val={min_val}")
                hdiff = max_val - min_val # if box_size = 1 - hdiff = 0 // Times 2 to account for 2m by 2m by 1m scale
                box_count = mt.ceil(hdiff / (2*box_size))
                box_count_for_size += box_count
                total_box_count += box_count
        # print(f"Total box count for box size {size} is {box_count_for_size}")
    return total_box_count

def fractal_dimension(tif_file: str, box_sizes: List[int], pdf: PdfPages):
    with rasterio.open(tif_file) as src:
        image = src.read(1)
    raster = np.array(image)

    counts = [box_count(raster, size) for size in box_sizes]

    log_eps = np.log(1 / np.array(box_sizes))
    log_counts = np.log(counts)

    slope, intercept = np.polyfit(log_eps, log_counts, 1)
    dateyear = tif_file.split('_')[3]
    fig, ax = plt.subplots(figsize=(8, 6))
    ax.plot(log_eps, log_counts, 'o', label='Log-Log Data')
    ax.plot(log_eps, slope * log_eps + intercept, 'r', label=f'Fit: slope = {slope:.4f}')
    ax.set_xlabel('log(1/ε)')
    ax.set_ylabel('log(N(ε))')
    ax.set_title(f'Fractal Dimension Estimation\n{dateyear}')
    ax.legend()
    ax.grid(True)

    pdf.savefig(fig)
    plt.close(fig)

    return slope

In [8]:
folder_path_relative = "updated-datasets"
list_of_box_sizes = [100, 1000]

with PdfPages("plots_of_fractal_dimensions.pdf") as pdf:
    for filename in os.listdir(folder_path_relative):
        file_path = os.path.join(folder_path_relative, filename)
        if os.path.isfile(file_path):
            fractal_dimension(tif_file=file_path, box_sizes=list_of_box_sizes, pdf=pdf)